In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# read tsv
df = pd.read_csv('gamail_orgnaizer_export_1701309047952.tsv', sep='\t')

# make account_role	all equal to 0
df['account_role'] = 0

# drop content, account_name
data = df.drop(['content', 'account_name', 'mail_labels', 'receiver'], axis=1)


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline

# Sample Data

data['combined_text'] = data['account_mail_address'] + ' ' + data['sender'] + ' ' + data['subject']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['combined_text'], data['account_role'], test_size=0.2, random_state=42)

# Create a pipeline that first converts text data to TF-IDF vectors and then applies Naive Bayes
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
report = classification_report(y_test, y_pred)

report

/Users/jackyzhang/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/jackyzhang/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/jackyzhang/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/jackyzhang/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` inste

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00        22\n\n    accuracy                           1.00        22\n   macro avg       1.00      1.00      1.00        22\nweighted avg       1.00      1.00      1.00        22\n'

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline

# Splitting the dataset into features (X) and target (y)
X = data[['account_mail_address', 'sender', 'subject']]
y = data['account_role']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# We will use a CountVectorizer to convert text data to numeric format
# It's important to note that this is a very basic form of text processing
# More complex techniques might be required for better performance

# Creating pipelines for each feature
pipeline_mail = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))
pipeline_sender = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))
pipeline_subject = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))

# Training the models
pipeline_mail.fit(X_train['account_mail_address'], y_train)
pipeline_sender.fit(X_train['sender'], y_train)
pipeline_subject.fit(X_train['subject'], y_train)

# Evaluating the models
score_mail = pipeline_mail.score(X_test['account_mail_address'], y_test)
score_sender = pipeline_sender.score(X_test['sender'], y_test)
score_subject = pipeline_subject.score(X_test['subject'], y_test)

score_mail, score_sender, score_subject



/Users/jackyzhang/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/jackyzhang/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Sample data - replace with your data
emails = ["Sample email texts..."]
labels = ["teacher", "student"]  # Labels

# Tokenization
tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

# Build Vocabulary
vocab = build_vocab_from_iterator(yield_tokens(emails), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Encode Labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Dataset Class
class EmailDataset(Dataset):
    def __init__(self, emails, labels):
        self.emails = emails
        self.labels = labels

    def __len__(self):
        return len(self.emails)

    def __getitem__(self, idx):
        email = torch.tensor(vocab(tokenizer(self.emails[idx])), dtype=torch.long)
        label = torch.tensor(self.labels[idx], dtype=torch.float)
        return email, label

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(emails, encoded_labels, test_size=0.2, random_state=42)

# DataLoader
train_data = EmailDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=1, shuffle=True)

# LSTM Model
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text, text_lengths):
        embedded = self.embedding(text)
        packed_embedded = pack_padded_sequence(embedded, text_lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        output, output_lengths = pad_packed_sequence(packed_output, batch_first=True)
        hidden = hidden.squeeze(0)
        return self.fc(hidden)

# Model Initialization
vocab_size = len(vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 1

model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)

# Training (simplified for example)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1):
    model.train()
    for emails, labels in train_loader:
        emails, labels = emails.to(device), labels.to(device)
        optimizer.zero_grad()
        predictions = model(emails, [len(email) for email in emails]).squeeze(1)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()

print("Training Complete")

# Add evaluation, saving/loading model, and inference as needed


  Obtaining dependency information for torchtext from https://files.pythonhosted.org/packages/f2/21/6934dc4820c2bb12c8ae2608a2456df31a2c0c58973f2278059757f35f8d/torchtext-0.16.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torch==2.1.1 from https://files.pythonhosted.org/packages/5b/46/3def5bdaae03c21a7662673e6bda1f60a046afce48e0d6319ce4542bca31/torch-2.1.1-cp311-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for torchdata==0.7.1 from https://files.pythonhosted.org/packages/35/b2/7ed3a80ae0673b940f2af14281dc02dee0f667c6094e6dcd399fa35249a7/torchdata-0.7.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 35.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 36.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 2.0.

ValueError: Found input variables with inconsistent numbers of samples: [1, 2]